In [3]:
from langchain_openai import ChatOpenAI
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain.prompts import PromptTemplate
import os 

In [4]:
openai_api_key = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(openai_api_key=openai_api_key)

graph = Neo4jGraph(
    url="neo4j://0.0.0.0:7687",
    username="neo4j",
    password="BRU0109al"
)

In [5]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
For movie titles that begin with "The", move "the" to the end, For example "The 39 Steps" becomes "39 Steps, The" or "The Matrix" becomes "Matrix, The".
If the user asks for a movie recommendation, return the movie with the highest rating.

Schema: {schema}
Question: {question}
"""

In [6]:
cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

In [7]:
cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True
)

In [12]:
cypher_chain.invoke({"query": "What is the first human message ever written in this schema?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Message)
RETURN m.content
ORDER BY m.id
LIMIT 1
Full Context:
[{'m.content': 'hello. how do see the movie scene in galicia these days? what are the most important works done so far?'}]

> Finished chain.


{'query': 'What is the first human message ever written in this schema?',
 'result': "I don't know the answer."}